In [1]:
import urllib.request, json 
import pandas as pd

In [14]:
#Create Api call example
#dataset_identifier="ICT_HH2"
#filter_dict={"Country":["FRA"],
#             "Indicator":[""], #Take all the indicators
#             "Breakdown":[""]} #Take all the breakdowns
#agency_name="all"
#params={"detail":"dataonly"}

In [3]:
def makeAPICall(dataset_identifier,
                filter_dict,
                agency_name,
                params={},
                OECD_BASE_URL="https://stats.oecd.org/SDMX-JSON/data/"):
    # See https://data.oecd.org/api/sdmx-json-documentation/#d.en.330346
    """
    Create the url for the OECD API
    Arguments:
        dataset_identifier: string containing the name of the dataset
        filter_dict: dictionary containing the value of the variables to query (Country can be only one)
        agency_name: agency name to pass to the api  
    """

    filter_expression=""
    filter_list=[]

    if all(filter_dict.values())=="":
        filter_expression="all"
    else:
        for dim in filter_dict.keys():
            filter_list.append("+".join(filter_dict[dim]))
    filter_expression=".".join(filter_list)
    
    api_string=f"{OECD_BASE_URL}{dataset_identifier}/{filter_expression}/{agency_name}"
    if len(params)>0:
        api_string+="?"
        for par in params.keys():
            api_string+=(par+'='+params[par])

    return api_string

In [4]:
#json_url=makeAPICall(dataset_identifier,filter_dict,agency_name,params)
#print(json_url)

In [5]:
def getData(json_url):
    "Return a dict with the data"
    
    with urllib.request.urlopen(json_url) as url:
        data = json.loads(url.read().decode())
        #print(data)
    return data

In [6]:
def convertToPandas(data):
    col_names=[x["id"] for x in data["structure"]["dimensions"]["observation"][0]["values"] ]
    #print(col_names)
     
    dataset_dimensions = data["structure"]["dimensions"]["series"]
    print(f"Dataset dimensions:\t{len(dataset_dimensions)}")
    
    country = dataset_dimensions[0]["values"][0]["id"]
    print(f"Country:\t{country}\n")
    
    indicators=[x["id"] for x in dataset_dimensions[1]["values"]]
    print(f"Indicators:\t{indicators}")
    print(f"N indicators = {len(indicators)}\n")
    
    breakdowns=[x["id"] for x in dataset_dimensions[2]["values"]]
    print(f"Breakdowns:\t{breakdowns}")
    print(f"N breakdowns = {len(breakdowns)}")
    
    df_ind_list=[]
    
    for k in data["dataSets"][0]["series"].keys():
        ind = int(k.split(':')[1])
        brkd = int(k.split(':')[2])
        df = pd.DataFrame(data["dataSets"][0]["series"][k]['observations'])
        df["Var"]=indicators[ind]
        df["Brkd"]=breakdowns[brkd]
        df.set_index(["Var","Brkd"],inplace=True)
        df_ind_list.append(df)
        
    
    df_res=pd.concat(df_ind_list,sort=False)
    df_res.columns = col_names
    
    df_res.reset_index(inplace=True)
    df_res["Country"]=country
    
    return df_res.set_index(["Country","Var","Brkd"])

In [7]:
#convertToPandas(data)

### Create final dataset

In [8]:
def createMultipleCountryDataset(country_list,
                                 dataset_identifier="ICT_HH2",
                                 indicator_list=[""],
                                 breakdown_list=[""],
                                 agency_name="all",                #do not touch
                                 params={"detail":"dataonly"}      #do not touch
                                ):
    """
    Create a dataset with multiple countries
    """

    filter_dict={
        "Country":[""], #leave it empty it will be overwritten by country list
        "Indicator":indicator_list,
        "Breakdown":breakdown_list
    }
    
    df_to_concat = []

    for country in country_list:
        filter_dict["Country"] = [country]
        json_url=makeAPICall(dataset_identifier,filter_dict,agency_name,params)
        print(json_url)
        data=getData(json_url)
        df_to_concat.append(convertToPandas(data))
    
    res = pd.concat(df_to_concat,sort=False)
    res.columns = sorted(res.columns)
    
    return res

### Multiple breakdown and indicator list is not working properly (quick workaround using loop)

In [37]:
# Specify here your list of indicators and breakdown 
country_list = ["DEU","ITA","FRA","NZA"]
indicator_list = ["A1","B1"]
breakdown_list = ["HH_TOTAL"]

df_list = []

for cou in country_list:
    for ind in indicator_list:
        for bkd in breakdown_list:
            try:
                df = createMultipleCountryDataset(
                        country_list = [cou], 
                        indicator_list = [ind],
                        breakdown_list = [bkd]
                )                
                df_list.append(df)
            except:
                raise ValueError(f"One of {(cou,ind,bkd)} not found")


# Union the various dataframes
df_final = pd.concat(df_list,sort=True)

https://stats.oecd.org/SDMX-JSON/data/ICT_HH2/DEU.A1.HH_TOTAL/all?detail=dataonly
Dataset dimensions:	3
Country:	DEU

Indicators:	['A1']
N indicators = 1

Breakdowns:	['HH_TOTAL']
N breakdowns = 1
https://stats.oecd.org/SDMX-JSON/data/ICT_HH2/DEU.B1.HH_TOTAL/all?detail=dataonly
Dataset dimensions:	3
Country:	DEU

Indicators:	['B1']
N indicators = 1

Breakdowns:	['HH_TOTAL']
N breakdowns = 1
https://stats.oecd.org/SDMX-JSON/data/ICT_HH2/ITA.A1.HH_TOTAL/all?detail=dataonly
Dataset dimensions:	3
Country:	ITA

Indicators:	['A1']
N indicators = 1

Breakdowns:	['HH_TOTAL']
N breakdowns = 1
https://stats.oecd.org/SDMX-JSON/data/ICT_HH2/ITA.B1.HH_TOTAL/all?detail=dataonly
Dataset dimensions:	3
Country:	ITA

Indicators:	['B1']
N indicators = 1

Breakdowns:	['HH_TOTAL']
N breakdowns = 1
https://stats.oecd.org/SDMX-JSON/data/ICT_HH2/FRA.A1.HH_TOTAL/all?detail=dataonly
Dataset dimensions:	3
Country:	FRA

Indicators:	['A1']
N indicators = 1

Breakdowns:	['HH_TOTAL']
N breakdowns = 1
https://stats.o

ValueError: One of ('NZA', 'A1', 'HH_TOTAL') not found

In [35]:
df_final

2005     2006     2007     2008     2009     2010  \
Country Var Brkd                                                             
DEU     A1  HH_TOTAL      NaN  76.8558  78.6466  81.8193  84.0926  85.7406   
        B1  HH_TOTAL  61.6383  67.1077  70.6745  74.8754  79.0690  82.4875   
ITA     A1  HH_TOTAL      NaN  51.5822  53.3796  55.9745  61.2675  64.8464   
        B1  HH_TOTAL  38.5781  40.0125  43.3831  46.9302  53.4823  58.9711   
FRA     A1  HH_TOTAL      NaN      NaN  65.5494  68.3557  74.2203  76.4499   
        B1  HH_TOTAL      NaN  40.9315  55.0468  62.3197  68.8683  73.6061   

                         2011     2012     2013     2014     2015     2016  \
Country Var Brkd                                                             
DEU     A1  HH_TOTAL  86.8648  87.0997  88.8744      NaN  90.9926      NaN   
        B1  HH_TOTAL  83.2549  85.4999  87.7470  89.4741  90.2891  92.1352   
ITA     A1  HH_TOTAL  66.1957  67.0906  71.1443      NaN  72.5436      NaN   
        B1  HH_TOTAL  61.5733  62.9165  68.9144  72.6141  75.3897  78.5100   
FRA     A1  HH_TOTAL  78.1792  81.0122  81.5595      NaN  81.5481      NaN   
        B1  HH_TOTAL  75.9151  80.0043  81.7192  82.9556  82.6207  85.8736   

                           2017       2018       2019  
Country Var Brkd                                       
DEU     A1  HH_TOTAL  92.865556        NaN        NaN  
        B1  HH_TOTAL  92.857134  94.393855  94.829303  
ITA     A1  HH_TOTAL  72.502047        NaN        NaN  
        B1  HH_TOTAL  81.024370  84.338355  85.172590  
FRA     A1  HH_TOTAL  84.124964        NaN        NaN  
        B1  HH_TOTAL  86.412793  88.562864  90.172666

### Write csv

In [125]:
df_final.to_csv("data.csv",header=True)